# Plan de Transformación de Datos para Modelado de Machine Learning

### Estudiantes
Amanda Alpízar Araya
Alonso Arias Mora
Danny Valverde Agüero

### Introducción y Objetivos

Este notebook detalla el proceso de transformación del conjunto de datos de calidad del agua, previamente limpiado en el notebook `EDA.ipynb`. El objetivo es convertir el DataFrame `df_cleaned` en un conjunto de datos completamente numérico, escalado y optimizado, listo para ser utilizado en algoritmos de machine learning, específicamente para tareas de clustering no supervisado como K-Means.

El proceso sigue un plan estructurado para garantizar que las transformaciones se apliquen en un orden lógico, maximizando la calidad de los datos para el modelado.

### Metodología de Transformación

El pipeline de transformación se ejecutará en el siguiente orden:

1.  **Selección de Características (Fase 1):** Se eliminarán columnas de alta cardinalidad, identificadores y variables categóricas redundantes. La columna `CONTAMINANTES` se conservará para ingeniería de características.
2.  **Codificación de Variables Categóricas:** Las variables categóricas se convertirán a formato numérico. Se usará **Multi-Label Binarization** para la columna `CONTAMINANTES` y **One-Hot Encoding** para las demás.
3.  **Transformación no Lineal:** Se aplicará una transformación logarítmica a las características numéricas para corregir el fuerte sesgo positivo en sus distribuciones.
4.  **Selección de Características (Fase 2):** Se analizará la matriz de correlación para identificar y eliminar características altamente correlacionadas, reduciendo la multicolinealidad.
5.  **Escalado de Datos:** Todas las características se escalarán utilizando estandarización para asegurar que todas contribuyan por igual al modelo.

### Resultado Final

El producto de este notebook será un DataFrame final, `df_scaled`, que contendrá únicamente datos numéricos y escalados, representando la versión más óptima del conjunto de datos para la fase de modelado.

### Sección 1: Configuración del Entorno y Carga de Datos Limpios

#### Resumen Ejecutivo de la Sección

Esta sección inicial configura el entorno de Python y carga el conjunto de datos `df_cleaned`, que es el resultado del proceso de limpieza del notebook `EDA.ipynb`. Se asume que este archivo ya ha sido generado y guardado. Realizaremos una inspección inicial para confirmar que los datos se cargan correctamente antes de comenzar con el pipeline de transformación.

In [ ]:
# --- Sección 1: Configuración del Entorno y Carga de Datos Limpios ---

# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler

# Ignorar advertencias para una salida limpia
warnings.filterwarnings("ignore")

# Configurar el estilo y tamaño de las gráficas
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# --- 1. Cargar el conjunto de datos limpio ---
# Este archivo es el resultado del notebook EDA.ipynb
cleaned_data_path = 'datos/df_cleaned.csv' 
try:
    # Asumimos que el df_cleaned fue guardado con su índice, por lo que lo usamos como index_col
    df_cleaned = pd.read_csv(cleaned_data_path, index_col=0)
    print(f"El conjunto de datos limpio se cargó exitosamente desde: '{cleaned_data_path}'.")
    print(f"Dimensiones del dataset: {df_cleaned.shape}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta: '{cleaned_data_path}'")
    print("Por favor, ejecute primero el notebook EDA.ipynb para generar este archivo.")
    df_cleaned = None

# --- 2. Mostrar información inicial del DataFrame ---
if df_cleaned is not None:
    print("\n--- Primeras 5 filas del conjunto de datos limpio ---")
    display(df_cleaned.head())
    print("\n--- Tipos de datos de las columnas ---")
    df_cleaned.info()

### Sección 2: Selección de Características (Fase 1)

#### Resumen Ejecutivo de la Sección

En esta sección, se realiza la primera fase de selección de características. El objetivo es eliminar columnas que no son adecuadas para el modelado, basándose en los hallazgos del EDA. Esto incluye la eliminación de identificadores únicos, variables geográficas de alta cardinalidad y columnas categóricas que son redundantes. **Crucialmente, la columna `CONTAMINANTES` se conserva para ser procesada en la siguiente etapa.**

In [ ]:
# --- Sección 2: Selección de Características (Fase 1) ---

if df_cleaned is not None:
    # Copiar el dataframe para mantener el original intacto
    df_features = df_cleaned.copy()

    # --- 1. Identificar columnas a eliminar ---
    
    # Columnas de identificadores únicos
    id_cols = ['CLAVE', 'SITIO']

    # Columnas categóricas de alta cardinalidad
    high_cardinality_cols = ['MUNICIPIO', 'ACUIFERO']

    # Columnas categóricas redundantes (calidad, cumplimiento, semáforo)
    # Se excluye 'CONTAMINANTES' de esta lista para conservarla
    redundant_categorical_cols = [col for col in df_features.columns if 'CALIDAD_' in col or 'CUMPLE_CON_' in col]
    redundant_categorical_cols.append('SEMAFORO')
    
    # Columna de PERIODO (constante, no aporta varianza)
    misc_cols_to_drop = ['PERIODO']

    # Combinar todas las columnas a eliminar
    cols_to_drop = id_cols + high_cardinality_cols + redundant_categorical_cols + misc_cols_to_drop
    # Asegurarse de que no haya duplicados en la lista
    cols_to_drop = list(set(cols_to_drop))

    # --- 2. Eliminar las columnas ---
    
    original_col_count = df_features.shape[1]
    df_features.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    new_col_count = df_features.shape[1]

    print(f"--- Selección de Características (Fase 1) ---")
    print(f"Se eliminaron {original_col_count - new_col_count} columnas.")
    print(f"El número de columnas pasó de {original_col_count} a {new_col_count}.")

    print("\n--- Columnas restantes en el DataFrame ---")
    display(df_features.head())
    df_features.info()

### Sección 3: Codificación de Variables Categóricas

#### Resumen Ejecutivo de la Sección

Esta sección convierte todas las variables categóricas en formato numérico. Se aplica un enfoque de dos pasos: primero, se utiliza **Multi-Label Binarization** en la columna `CONTAMINANTES` para crear una característica binaria para cada tipo de contaminante. Segundo, se aplica **One-Hot Encoding** a las demás variables categóricas de baja cardinalidad.

In [ ]:
# --- Sección 3: Codificación de Variables Categóricas ---

if 'df_features' in locals():
    df_encoded = df_features.copy()
    
    # --- 1. Codificación Multi-Label para 'CONTAMINANTES' ---
    print("--- Codificación Multi-Label para 'CONTAMINANTES' ---")
    
    # Limpiar el texto y dividirlo en una lista de contaminantes
    # Se reemplaza el valor de no-contaminación para que no se divida
    df_encoded['CONTAMINANTES'] = df_encoded['CONTAMINANTES'].str.replace('Sin Contaminantes', 'NINGUNO')
    
    # Crear variables dummy a partir de las etiquetas separadas por coma
    contaminant_dummies = df_encoded['CONTAMINANTES'].str.get_dummies(sep=', ')
    
    # Renombrar columnas para claridad
    contaminant_dummies.columns = [f'CONT_{col.replace(" ", "_")}' for col in contaminant_dummies.columns]
    print(f"Se crearon {contaminant_dummies.shape[1]} nuevas características desde 'CONTAMINANTES'.")

    # Unir las nuevas columnas y eliminar la original
    df_encoded = pd.concat([df_encoded, contaminant_dummies], axis=1)
    df_encoded.drop('CONTAMINANTES', axis=1, inplace=True)
    
    # --- 2. One-Hot Encoding para el resto de categóricas ---
    categorical_cols = df_encoded.select_dtypes(include=['object', 'category']).columns
    if not categorical_cols.empty:
        print(f"\n--- Codificación One-Hot para {len(categorical_cols)} variables restantes ---")
        print("Columnas a codificar:", list(categorical_cols))
        df_encoded = pd.get_dummies(df_encoded, columns=categorical_cols, drop_first=True)
    else:
        print("\nNo hay otras columnas categóricas para codificar.")

    print(f"\nEl número total de columnas es ahora {df_encoded.shape[1]}.")
    print("\n--- DataFrame después de la Codificación Completa ---")
    display(df_encoded.head())
    df_encoded.info()

### Sección 4: Transformación no Lineal (Logarítmica)

#### Resumen Ejecutivo de la Sección

En esta sección, se aborda el fuerte sesgo positivo detectado en las distribuciones de las variables de medición. Se aplicará una transformación logarítmica (`log(1+x)`) para normalizar estas distribuciones. Esta transformación comprime el rango de los valores grandes y expande el de los valores pequeños, haciendo que la distribución sea más simétrica. Esto es crucial para el rendimiento de algoritmos sensibles a la escala y la distribución, como K-Means y PCA.

In [ ]:
# --- Sección 4: Transformación no Lineal (Logarítmica) ---

if 'df_encoded' in locals():
    # Identificar las columnas de mediciones originales (excluyendo coordenadas y columnas ya codificadas)
    measurement_cols = df_cleaned.select_dtypes(include=np.number).columns
    # Excluir latitud y longitud, ya que no suelen requerir esta transformación
    cols_to_transform = [col for col in measurement_cols if col in df_encoded.columns and col not in ['LATITUD', 'LONGITUD']]
    
    print(f"--- Aplicando Transformación Logarítmica a {len(cols_to_transform)} columnas ---")
    
    # Crear una copia para la transformación
    df_transformed = df_encoded.copy()

    # Visualización Antes y Después para una columna de ejemplo
    example_col = 'CONDUCT_mS/cm'
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(df_transformed[example_col], kde=True)
    plt.title(f'Distribución de {example_col} (Antes de la Transformación)')

    # Aplicar la transformación log1p a todas las columnas seleccionadas
    for col in cols_to_transform:
        df_transformed[col] = np.log1p(df_transformed[col])

    plt.subplot(1, 2, 2)
    sns.histplot(df_transformed[example_col], kde=True, color='green')
    plt.title(f'Distribución de {example_col} (Después de la Transformación Log)')
    plt.show()
    
    print("\nTransformación logarítmica completada.")

### Sección 5: Selección de Características (Fase 2 - Correlación)

#### Resumen Ejecutivo de la Sección

Ahora que los datos son numéricos y sus distribuciones han sido mejoradas, se procede a la segunda fase de selección de características. El objetivo es identificar y eliminar la multicolinealidad, que ocurre cuando dos o más características están altamente correlacionadas. Se calculará una matriz de correlación y se eliminará una de cada par de características cuya correlación supere un umbral predefinido (e.g., 0.90), conservando así la que aparezca primero en el DataFrame.

In [ ]:
# --- Sección 5: Selección de Características (Fase 2 - Correlación) ---

if 'df_transformed' in locals():
    # Calcular la matriz de correlación
    corr_matrix = df_transformed.corr().abs()

    # Visualizar el heatmap de correlación
    plt.figure(figsize=(24, 20))
    sns.heatmap(corr_matrix, cmap='viridis', annot=False)
    plt.title('Heatmap de Correlación de Características Transformadas')
    plt.show()

    # Identificar y eliminar características altamente correlacionadas
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop_corr = [column for column in upper_tri.columns if any(upper_tri[column] > 0.90)]

    print(f"--- Análisis de Correlación (Umbral > 0.90) ---")
    if to_drop_corr:
        print(f"Se encontraron {len(to_drop_corr)} columnas para eliminar debido a alta correlación:")
        print(to_drop_corr)
        df_reduced = df_transformed.drop(columns=to_drop_corr)
        print(f"\nEl número de columnas se redujo de {df_transformed.shape[1]} a {df_reduced.shape[1]}.")
    else:
        print("No se encontraron características con una correlación superior al umbral.")
        df_reduced = df_transformed.copy()

    print("\n--- DataFrame después de eliminar alta correlación ---")
    display(df_reduced.head())

### Sección 6: Escalado de Datos (Estandarización)

#### Resumen Ejecutivo de la Sección

Este es el paso final de la preparación de datos. Las características, aunque transformadas, todavía se encuentran en diferentes escalas. Para asegurar que los algoritmos de clustering (como K-Means) traten a todas las características con la misma importancia, se aplica la estandarización. Este proceso reescala cada característica para que tenga una media de 0 y una desviación estándar de 1. El resultado es el DataFrame final, `df_scaled`, listo para el modelado.

In [ ]:
# --- Sección 6: Escalado de Datos (Estandarización) ---

if 'df_reduced' in locals():
    # Inicializar el escalador
    scaler = StandardScaler()

    # Aplicar el escalador
    scaled_features = scaler.fit_transform(df_reduced)

    # Convertir el resultado de nuevo a un DataFrame
    df_scaled = pd.DataFrame(scaled_features, index=df_reduced.index, columns=df_reduced.columns)

    print("--- Escalado de Datos Completado ---")
    print("Todas las características han sido estandarizadas (media=0, std=1).")
    
    print("\n--- Vista Previa del DataFrame Final Escalado ---")
    display(df_scaled.head())

    print("\n--- Verificación de la media y desviación estándar después del escalado ---")
    display(df_scaled.describe().round(2))

### Sección 7: Resumen y Conclusiones Finales

#### Resumen de Transformaciones

El conjunto de datos ha sido sometido a un riguroso pipeline de transformaciones para prepararlo para el modelado de machine learning. Las operaciones realizadas fueron:

1.  **Selección de Características (Fase 1):** Se eliminaron columnas de identificadores, alta cardinalidad y características derivadas redundantes, conservando `CONTAMINANTES`.
2.  **Codificación:** La columna `CONTAMINANTES` se expandió a 17 características binarias. Las variables `ORGANISMO_DE_CUENCA` y `SUBTIPO` se codificaron en 14 columnas. 
3.  **Transformación Logarítmica:** Se aplicó `np.log1p` a 14 variables de medición para normalizar sus distribuciones.
4.  **Selección de Características (Fase 2):** Se eliminó 1 columna (`SDT_M_mg/L`) debido a su alta correlación (>0.90) con `CONDUCT_mS/cm`.
5.  **Estandarización:** Todas las características finales fueron escaladas a una media de 0 y una desviación estándar de 1.

#### Conclusión

El resultado final es el DataFrame `df_scaled`. Este conjunto de datos es numérico, no tiene valores faltantes, presenta distribuciones mejoradas, está libre de multicolinealidad severa y todas sus características están en la misma escala.

El conjunto de datos está ahora en condiciones óptimas para ser utilizado en algoritmos de clustering como K-Means o para análisis de componentes principales (PCA), que sería un siguiente paso lógico para explorar la estructura latente de los datos.